In [1]:
import numpy as np
#import scipy
from scipy import linalg

- https://stackoverflow.com/questions/64218559/simple-linear-dependency
- https://en.wikipedia.org/wiki/Linear_independence
- https://en.wikipedia.org/wiki/LU_decomposition
- https://en.wikipedia.org/wiki/Gaussian_elimination


In [2]:
v1 = np.array([0, 5, 0])
v2 = np.array([0, -10, 0])
v3 = np.array([1, 2, 3])
v4 = np.array([-2, -4, -6])

In [3]:
p, l, u = linalg.lu(np.array([v1, v3]).T)
p, l, u

(array([[0., 0., 1.],
        [1., 0., 0.],
        [0., 1., 0.]]),
 array([[1.        , 0.        ],
        [0.        , 1.        ],
        [0.        , 0.33333333]]),
 array([[5., 2.],
        [0., 3.]]))

In [4]:
np.linalg.matrix_rank(np.array([v1, v2]).T)

1

In [5]:
np.linalg.matrix_rank(np.array([v1, v3]).T)

2

In [6]:
np.cross(v1, v2)

array([0, 0, 0])

In [7]:
np.cross(v1, v3)

array([15,  0, -5])

In [8]:
U, s, V = linalg.svd(np.array([v1, v2]).T)
s

array([11.18033989,  0.        ])

In [9]:
U, s, V = linalg.svd(np.array([v1, v3]).T)
s

array([5.55992016, 2.84381571])

In [10]:
def indep1(*args):
    A = np.array(args).T
    return np.linalg.matrix_rank(A) == len(args)

In [11]:
indep1(v1, v2, v3)

False

In [12]:
def indep2(a, b):
    return not np.allclose(np.cross(a, b), 0.)

In [13]:
indep2(v1, v2)
indep2(v1, v3)
#indep2(v1, v2, v3)

True

In [14]:
#np.linalg.solve(np.array([v1, v2, v3]).T, np.zeros(v1.size))

In [15]:
def indep3(a, b):
    r = a/b
    q = np.isfinite(r)
    return not np.allclose(r[q], r[q][0])
    #return np.allclose(r, r[0])

In [16]:
indep3(v1, v2)
#indep3(v1, v3)
#indep3(v3, v4)
#indep3(v3, np.zeros(v1.size)) # IndexError

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


False

In [17]:
indep1(v1, np.zeros(v1.size))

False

In [18]:
indep2(v1, np.zeros(v1.size))

False

In [19]:
def indep4(a, b):
    return not np.isclose(np.dot(a,b)*np.dot(b,a), np.dot(a,a)*np.dot(b,b))

In [20]:
indep4(v1, v2)
indep4(v3, v4)
indep4(v1, v4)
indep4(v3, np.zeros(v1.size)) # IndexError

False

In [21]:
def is_linearly_dependent(a, b):
    non_zero = b != 0
    if len(np.unique(a[non_zero]/b[non_zero])) > 1:
        return False
    else:
        zero = np.logical_not(b)
        if np.any(a[zero] != 0):
            return False
    return True

def indep5(a, b):
    return not is_linearly_dependent(a, b)

In [22]:
eps = np.finfo(np.float64).eps
eps

2.220446049250313e-16

In [23]:
funcs = [indep1, indep2, indep3, indep4, indep5]
vects = {
    "0": (0, 0, 0),
    "0+eps": (eps, eps, eps),
    "1": (1., 1., 1.),
    "1.1": (1.1, 1, 1),
    "-1": (-1,-1,-1),
    "1+eps": (1, 1 ,1+eps),
    "1e10": (1e10, 1e10, 1e10),
    "1e20": (1e20, 1e20, 1e20)
}

In [24]:
5*len(vects)**2

320

In [25]:
import itertools
import pandas as pd

In [26]:
def check_symmetric(a, rtol=1e-05, atol=1e-08):
    return np.allclose(a, a.T, rtol=rtol, atol=atol)

In [27]:
def apply(vectors, functions):
    results = []
    for (k1, k2) in itertools.product(vectors, vectors):
        v1 = np.array(vectors[k1])
        v2 = np.array(vectors[k2])
        for f in functions:
            try:
                res = int(f(v1, v2))
            except:
                res = -1
            results.append({
                'function': f.__name__,
                'k1': k1, 'k2': k2,
                'v1': v1, 'v2': v2,
                'result': res
            })
    return pd.DataFrame(results)

In [28]:
def check(frame):
    c = frame.pivot_table(index='k1', columns=['function', 'k2'], values='result')
    print(c.columns.levels[0])
    res = []
    for k in c.columns.levels[0]:
        x = c[k].values
        res.append({
            "function": k,
            "symmetric": check_symmetric(x),
            "self-call": np.allclose(np.diag(x), 0.)
        })
    return pd.DataFrame(res)

In [29]:
pd.options.display.max_rows=200

In [30]:
df = apply(vects, funcs)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in true_divide
  


In [31]:
df.pivot_table(index=['function', 'k1'], columns='k2', values='result')

k2              -1  0  0+eps  1  1+eps  1.1  1e10  1e20
function k1                                            
indep1   -1      0  0      0  0      0    1     0     0
         0       0  0      0  0      0    0     0     0
         0+eps   0  0      0  0      0    0     0     0
         1       0  0      0  0      0    1     0     0
         1+eps   0  0      0  0      0    1     0     0
         1.1     1  0      0  1      1    0     1     0
         1e10    0  0      0  0      0    1     0     0
         1e20    0  0      0  0      0    0     0     0
indep2   -1      0  0      0  0      0    1     0     0
         0       0  0      0  0      0    0     0     0
         0+eps   0  0      0  0      0    0     0     0
         1       0  0      0  0      0    1     0     0
         1+eps   0  0      0  0      0    1     1     1
         1.1     1  0      0  1      1    0     1     1
         1e10    0  0      0  0      1    1     0     0
         1e20    0  0      0  0      1    1     0     0
indep3   -1      0 -1      0  0      0    1     0     0
         0       0 -1      0  0      0    0     0     0
         0+eps   0 -1      0  0      0    0     0     0
         1       0 -1      0  0      0    1     0     0
         1+eps   0 -1      0  0      0    1     0     0
         1.1     1 -1      1  1      1    0     0     0
         1e10    0 -1      0  0      0    1     0     0
         1e20    0 -1      0  0      0    1     0     0
indep4   -1      0  0      0  0      0    1     0     0
         0       0  0      0  0      0    0     0     0
         0+eps   0  0      0  0      0    0     0     0
         1       0  0      0  0      0    1     0     0
         1+eps   0  0      0  0      0    1     0     0
         1.1     1  0      0  1      1    0     1     1
         1e10    0  0      0  0      0    1     0     0
         1e20    0  0      0  0      0    1     0     0
indep5   -1      0  1      0  0      1    1     0     0
         0       0  0      0  0      0    0     0     0
         0+eps   0  1      0  0      1    1     0     0
         1       0  1      0  0      1    1     0     0
         1+eps   1  1      1  1      0    1     1     1
         1.1     1  1      1  1      1    0     1     1
         1e10    0  1      0  0      1    1     0     0
         1e20    0  1      0  0      1    1     0     0

In [32]:
check(df)

Index(['indep1', 'indep2', 'indep3', 'indep4', 'indep5'], dtype='object', name='function')


,function,symmetric,self-call
0,indep1,True,True
1,indep2,True,True
2,indep3,False,False
3,indep4,True,True
4,indep5,False,True


In [33]:
v1 = np.array([1,1,1])
v2 = -v1

In [34]:
indep5(v1, v2)

False

In [35]:
indep5(v2, v1)

False

In [36]:
indep5(np.array((0,0,0)), v1)

False

In [37]:
indep5(v4, np.array((0,0,0)))

True

In [38]:
indep5(v1, v1+eps)

False

In [39]:
indep5(v1, v1)

False

In [40]:
indep5(v1, v1)

False

In [41]:
indep5(v4, v4+eps)

True

In [42]:
indep5(v4+eps, v4)

True

In [43]:
z = np.array([0,0,0])

In [44]:
is_linearly_dependent(z, v1)

True

In [45]:
is_linearly_dependent(v1, z)

False

In [46]:
is_linearly_dependent(v1, v1+eps)

True

In [47]:
is_linearly_dependent(v4, v4+eps)

False

Notice your algorithm is not symmetrical: `False == f(0,v) != f(v,0) == True`. And off course it fails to cope with float arithmetic inaccuracies depending on the vector `f(v1, v1+eps) == True` or `f(v2, v2+eps) == False`.

First issue is float arithmetic: this algorithm considers any v and v+eps (where eps is machine precision) as different vectors which is generally a problem when dealing with numerical Linear Algebra. Additionally this algorithm also fails with simple vectors such as v=(1,1,1) and -v which are stated to be linearly independent. It also says that 0 and v are linearly independent when 0 is the LHS variable and the converse when swapped. I'll suggest you to test it deeper with well known cases.